In [ ]:
from google.colab import drive
import numpy as np
import math
import random
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from scipy.optimize import linear_sum_assignment
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import pickle, os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_dataset(name_file):

    desired_directory = '/content/drive/MyDrive/project' # Replace with your desired directory path

    file_path = os.path.join(desired_directory, name_file)

    with open(file_path, 'rb') as f:
        data_dict = pickle.load(f)

    return data_dict

In [ ]:
loaders_dict = load_dataset(f"dataset-flowers102-features.pkl")
x_train = loaders_dict["x_train"]
x_test = loaders_dict["x_test"]
y_train = loaders_dict["y_train"]
y_test = loaders_dict["y_test"]
print(f"x_train:{x_train.shape}, y_train:{y_train.shape}")
print(f"x_test:{x_test.shape}, y_test:{y_test.shape}")

In [ ]:
cluster = KMeans(n_clusters=50, n_init=10, random_state=42).fit(x_train)
# cluster = KMeans(n_clusters=50, n_init=10, random_state=42, algorithm='auto', precompute_distances='manhattan').fit(x_train)
# cluster = KMeans(n_clusters=50, n_init=10, random_state=42,metric=distance_metric('manhattan')).fit(x_train)
labelsDB = cluster.labels_
centers = cluster.cluster_centers_


In [ ]:
def groupByLabels(data_labels,kmean_labels):
  group={}
  vec_to_group={}
  for vec,label in zip(data_labels,kmean_labels):
    if(label not in group.keys()):
      group[label]=[]
    group[label].append(vec)

  return group
group=groupByLabels(y_train,labelsDB)

In [ ]:
def purityy(group):
  sum=0;
  list_of_purity=[]
  clusts=0
  label_of_clust=[]
  for clust in group.values():
    max_label=Counter(clust).most_common(1)[0]
    label_of_clust.append(max_label[0])
    # print(max_label)
    list_of_purity.append((len(clust),max_label[1]/len(clust)))

    clusts+=len(clust)
    sum+=max_label[1]
  return list_of_purity,sum/clusts,label_of_clust


list_of_purity,sum,label_of_clust=purityy(group)
print(f"list_of_purity{list_of_purity}")
print(f"sum_of_purity{sum}")
print(f"label_of_cluster{label_of_clust}")


In [ ]:
from itertools import combinations
from sklearn.metrics import rand_score

def rand_index(cluster1, cluster2):
    """
    Computes the Rand Index between two clusterings.

    Arguments:
    cluster1 -- List or array containing the first clustering
    cluster2 -- List or array containing the second clustering

    Returns:
    rand_index -- Rand Index value between the two clusterings
    """
    if len(cluster1) != len(cluster2):
        raise ValueError("Both clusterings must have the same length")

    n = len(cluster1)
    a = 0  # Number of pairs that are in the same cluster in both clusterings
    b = 0  # Number of pairs that are in different clusters in both clusterings

    for i in range(n-1):
        for j in range(i+1, n):

            # print(f'c1:{cluster1[i]}, {cluster1[j]}')
            # print(f'c2:{cluster2[i]}, {cluster2[j]}')
            if cluster1[i] == cluster1[j] and cluster2[i] == cluster2[j]:
                a += 1
            elif cluster1[i] != cluster1[j] and cluster2[i] != cluster2[j]:
                b += 1
    total=n * (n - 1) / 2

    print(f"TP:{a},TN:{b},total={total}")
    rand_index = (a + b) / (n * (n - 1) / 2)
    return rand_index

def predict_label(data):
  lists=[]
  for i in data:
    lists.append( label_of_clust[i])
  return lists


print('rand index=' ,rand_index(y_train,labelsDB ))


In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


# kmeans = KMeans(n_clusters=3)  # Change the number of clusters as needed
# kmeans.fit(x_train)

pca = PCA(3)  # We specify 2 to indicate the number of dimensions we want
data_pca = pca.fit_transform(x_train)

plt.scatter(data_pca[:, 0], data_pca[:, 1], data_pca[:, 2] ,c=labelsDB)
plt.show()



In [ ]:
# Reduce the dimensionality of the data to 3D using PCA
pca = PCA(n_components=3)
vectors_reduced = pca.fit_transform(x_train)

# Plot the data points and the cluster centroids
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
sctt =ax.scatter(vectors_reduced[:, 0], vectors_reduced[:, 1], vectors_reduced[:,2] , c=labelsDB ,cmap='gist_rainbow' )
# centroids_reduced = pca.transform(cluster.cluster_centers_)
# ax.scatter(centroids_reduced[:, 0], centroids_reduced[:, 1], centroids_reduced[:, 2], marker='x', color='black')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
vectors_reduced = tsne.fit_transform(x_train)

# Plot the data points and the cluster centroids
plt.scatter(vectors_reduced[:, 0], vectors_reduced[:, 1], c=labelsDB ,cmap='gist_rainbow')

plt.show()

In [ ]:
tsne = TSNE(n_components=3, random_state=0)
vectors_reduced = tsne.fit_transform(x_train)

# Plot the data points and the cluster centroids
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
sctt =ax.scatter(vectors_reduced[:, 0], vectors_reduced[:, 1],vectors_reduced[:, 2], c=labelsDB ,cmap='gist_rainbow')
plt.show()

In [ ]:

def calculate_distance(x_test ,centers):
  distance_list=[]
  for  i,test in enumerate(x_test):
      distance_list.append([])
      for index,cent in enumerate(centers):
          dist = np.linalg.norm(test - cent)
          distance_list[i].append((index, dist))
      distance_list[i].sort(key=lambda x: x[1])
  return distance_list

In [ ]:
# distance_list=np.load('/content/drive/MyDrive/project/distances.npy')
distance_list=calculate_distance(x_test,centers)

np.save('/content/drive/MyDrive/project/distances',distance_list)

In [ ]:
def find_most_common_elements(data_list):
    counter = Counter(data_list)
    max_count = max(counter.values())
    most_common = [k for k, v in counter.items() if v == max_count]
    return most_common

# a=[3,4,5,6,3,3,4,5,5,6,6,4]
# find_most_common_elements(a)

# data = [1, 2, 3, 4, 2, 3, 2, 4, 4, 4]
# most_common_elements = find_most_common_elements(data)
# print("Most common elements:", most_common_elements)

In [ ]:
def meanbylabel(data):
  group={}
  lists={}
  for index,dis in data:
    label=y_train[index]
    if(label not in group.keys()):
      group[label]=[]
    group[label].append(dis)
    # vec_to_group[f'{vec}']=label
  for i in group.keys():
    lists[i]=np.mean(group[i])
  return lists

# d=[(0,3),(1,4),(2,5),(3,7)]
# groupbylabel(d)

In [ ]:
def get_nearest(datapoint, k):
  dist = cluster.transform([datapoint])
  result=[x for  x in enumerate(dist[0])]
  # print(result)
  result = sorted(result, key=lambda x: x[1], reverse=False)
  # print(result)
  return [i[0] for i in result[:k]]


get_nearest(x_test[0],3)

[44, 0, 15]

In [ ]:
# neigh = KNeighborsClassifier(n_neighbors=3)
# neigh.fit(x_train, y_train)
# y_pred=neigh.predict(x_test)
# acc=accuracy_score(y_test,y_pred)
# acc

In [ ]:
info={}
def save(n,k,scores):
  info['n']=n
  info['k']=k
  info['scores']=scores
  with open('/content/drive/MyDrive/project/scores2.pkl', 'wb') as handle:
      pickle.dump(info, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load():
  with open('/content/drive/MyDrive/project/scores.pkl', 'rb') as handle:
      info = pickle.load(handle)
  return info['n'], info['k'],info['scores']

In [ ]:
start_n,start_k,scores=load()

In [ ]:
# from sklearn.neighbors import DistanceMetric

for n in range(start_n,10):
  for k in range(6,10):
    predict_labels=[]
    min=0
    for i in range(len(x_test)):
      point_list=[]
      list_of_data=[]
      list_of_labels=[]
      selected_clusters= get_nearest(x_test[i],n)#[i[0] for i in distance_list[i][:n]]
      for index,cluster_label in enumerate(labelsDB):
        if(cluster_label in selected_clusters) :
          list_of_data.append(x_train[index])
          list_of_labels.append(y_train[index])
      np.random.seed(42)
      # dist = DistanceMetric.get_metric('manhattan')
      # knn = NearestNeighbors(n_neighbors=1, metric=dist)
      neigh = KNeighborsClassifier(n_neighbors=k , metric='manhattan')
      neigh.fit(list_of_data, list_of_labels)
      y_pred=neigh.predict([x_test[i]])
      predict_labels.append(y_pred[0])
      if(y_pred!=y_test[i])
    acc=accuracy_score(y_test, predict_labels)

    score=(acc*100)-(n*(0.2))
    scores.append((n,k,score))

    # purity = purity_score(y_test, predict_labels)
    # rand_index_value = rand_index(y_test, predict_labels)

    # print(f'Purity: {purity:.4f}')
    # print(f'Rand Index: {rand_index_value:.4f}')

    print(f'n={n},,k={k}, sc={score}, acc={acc}')
    save(n,k,scores)
  print("---------------------------")

print(scores)

n=9,,k=6, sc=83.206105006105, acc=0.85006105006105
n=9,,k=7, sc=83.3037851037851, acc=0.851037851037851
n=9,,k=8, sc=82.88864468864469, acc=0.8468864468864469
n=9,,k=9, sc=83.05958485958486, acc=0.8485958485958486
---------------------------
[(1, 1, 80.04420024420024), (1, 2, 78.798778998779), (1, 3, 79.06739926739927), (1, 4, 79.36043956043956), (1, 5, 78.798778998779), (1, 6, 78.65225885225885), (1, 7, 78.23711843711844), (1, 8, 78.04175824175825), (1, 9, 77.87081807081807), (2, 1, 81.82222222222221), (2, 2, 80.43028083028082), (2, 3, 81.77338217338216), (2, 4, 82.2129426129426), (2, 5, 82.1152625152625), (2, 6, 81.70012210012209), (2, 7, 81.96874236874235), (2, 8, 81.79780219780218), (2, 9, 81.45592185592184), (3, 1, 82.28156288156289), (3, 2, 80.81636141636142), (3, 3, 82.81880341880343), (3, 4, 82.9897435897436), (3, 5, 83.40488400488401), (3, 6, 83.28278388278389), (3, 7, 83.18510378510379), (3, 8, 82.96532356532357), (3, 9, 82.7943833943834), (4, 1, 82.32576312576313), (4, 2, 80

In [ ]:
def get_unique(tuple_list):
  result=[]
  for t in tuple_list:
    if(t not in result):
      result.append(t)
  return result

scores=get_unique(scores)

In [ ]:
scores.sort(key=lambda x: x[2], reverse=True)
max_score=scores[0]
scores

In [ ]:
max_score

(4, 5, 83.86422466422466)

In [ ]:
def filter(condition,value,output,sortBy='score' ):
  s={'n':0,'k':1,'score':2}
  condition_index,target_index,sort_index= s[condition],s[output],s[sortBy]
  scores.sort(key=lambda x: x[sort_index], reverse=True)

  result=[]
  my_map={}
  for x in scores:
    if(value==-1 or x[condition_index]==value ):
        if(x[0]  not in my_map):
          my_map[x[0]]={}
        if(x[1] not in my_map[x[0]]):
          my_map[x[0]][x[1]]=0;
        my_map[x[0]][x[1]]=max( my_map[x[0]][x[1]],x[target_index])
  for v in my_map.values():
    for m in v.values():
      result.append(m)
  return result

In [ ]:
n_list=filter('k',5,'n',sortBy='n')
k_list=filter('n',5,'k',sortBy='k')
n_score_list=filter('k',5,'score',sortBy='n')
k_score_list=filter('n',5,'score',sortBy='k')

In [ ]:
plt.plot(n_list,n_score_list)
plt.xlabel("n (k=5)")
plt.ylabel("scores")

In [ ]:
plt.plot(k_list,k_score_list)
plt.xlabel("k (n=5)")
plt.ylabel("scores")

In [ ]:
all_score_list=filter('n',-1,'score')
all_n_list=filter('k',-1,'n')
all_k_list=filter('n',-1,'k')
fig = plt.figure(figsize = (10, 7))


In [ ]:
all_score_list=filter('n',-1,'score')
all_n_list=filter('k',-1,'n')
all_k_list=filter('n',-1,'k')
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
sctt =ax.scatter(all_n_list,all_k_list,all_score_list,cmap='gist_rainbow')
ax.plot(max_score[0], max_score[1],max_score[2],'or')
ax.set_xlabel("n")
ax.set_ylabel("k")
plt.show()

In [ ]:
arr=np.zeros((9,9))
for i,j,k in zip(all_n_list,all_k_list,all_score_list):
  arr[i-1][j-1]=k;


In [ ]:
import seaborn as sns
ax= sns.heatmap(arr,fmt=".2f", annot=True,xticklabels=range(1,10),yticklabels=range(1,10),cmap=sns.color_palette("flare", as_cmap=True))
ax.set(xlabel="k", ylabel="n")


In [ ]:
def check_clusters(clusters,flag=False):
  count=0;
  for c in clusters:
    purity=list_of_purity[c][1]
    if flag:
       if(purity>0.7):
          count+=1
    else:
        if(purity<0.4):
          count+=1
  return count>2

k,n=4,5
labels=[]
min=0
true_count=0
false_count=0
total_bad=0
total_good=0
true_good=0
false_bad=0

for i in range(len(x_test)):
  print(f'i={i}')
  point_list=[]
  list_of_data=[]
  list_of_labels=[]
  selected_clusters= get_nearest(x_test[i],n)#[i[0] for i in distance_list[i][:n]]
  for index,cluster_label in enumerate(labelsDB):
    if(cluster_label in selected_clusters) :
      list_of_data.append(x_train[index])
      list_of_labels.append(y_train[index])
  np.random.seed(42)
  # dist = DistanceMetric.get_metric('manhattan')
  # knn = NearestNeighbors(n_neighbors=1, metric=dist)
  neigh = KNeighborsClassifier(n_neighbors=k , metric='manhattan')
  neigh.fit(list_of_data, list_of_labels)
  y_pred=neigh.predict([x_test[i]])
  isGood=check_clusters(selected_clusters,True)
  isBad=check_clusters(selected_clusters)
  if(isGood):
    total_good+=1
  elif (isBad):
    total_bad+=1

  if(y_pred==y_test[i]):
    true_count+=1
    if(isGood):
      true_good+=1
  else:
    false_count+=1
    if(isBad):
      false_bad+=1

In [ ]:
print(f'select good clusters:{total_good}')
print(f'true:{100*true_good/total_good} %')
print(f'select bad clusters:{total_bad}')
print(f'mistakes:{100*false_bad/total_bad} %')
print(true_good/true_count)
print(false_bad/false_count)

In [ ]:
        # dist = np.linalg.norm(x_test[i] - x_train[index] )
        # point_list.append((index,dist))
    # point_list.sort(key=lambda x: x[1],reverse=True)
    # n2=int(len(list_of_data)/n)
    # neigh = KNeighborsClassifier(n_neighbors=n2)
    # neigh.fit(list_of_data, list_of_labels)
    # label_list=[]
    # for selected_point in point_list[:n2] :#second n
    #   label_list.append(y_train[selected_point[0]])
    # most_common_elements = find_most_common_elements(label_list)
    # lists=meanbylabel(point_list)
    # if len(most_common_elements) ==1 :
    #   predict_labels.append(most_common_elements[0])
    # else :
    #   dist_in_list=[]
    #   for m in lists.keys():
    #     if m in most_common_elements:
    #       dist_in_list.append(lists[m])
    #   min=np.argmin(dist_in_list)
    #   print(min)
    #   predict_labels.append(most_common_elements[min])